# BIOMEDIN 260/RAD260: Problem Set 3 - Mammogram Project

## Spring 2021

## Name 1:

Aaron Sossin

## Name 2:

Vivian Zhu

## Introduction

Breast cancer has the highest incidence and second highest mortality rate for women in the US.

Your task is to utilize machine learning to study mammograms in any way you want (e.g. classification, segmentation) as long as you justify why it is useful to do whatever it is you want to do. Turning in a deep dream assignment using mammograms might be amusing, for example, but not so useful to patients. That being said, choose something that interests you. As the adage goes, "do what you love, and you’ll never have to work another day in your life, at least in BMI 260."

Treat this as a mini-project. We highly encourage working with 1 other person, possibly someone in your main project team. 

In addition to the mammograms themselves, the dataset includes "ground-truth" segmentations and `mass_case_description_train_set.csv`, which contains metadata information about mass shapes, mass margins, assessment numbers, pathology diagnoses, and subtlety in the data. Take some time to research what all of these different fields mean and how you might utilize them in your work. You dont need to use all of what is provided to you.

Some ideas:

1. Use the ROI’s or segmentations to extract features, and then train a classifier based on those features using the algorithms presented to you in the machine learning lectures (doesn't need to use deep learning).

2. Use convolutional neural networks. Feel free to use any of the code we went over in class or use your own (custom code, sklearn, keras, Tensorflow etc.). If you dont want to place helper functions and classes into this notebook, place them in a `.py` file in the same folder called `helperfunctions.py` and import them into this notebook.

## Data

The data is here:

https://wiki.cancerimagingarchive.net/display/Public/CBIS-DDSM

## Grading and Submission

This assignment has 3 components: code, figures (outputs/analyses of your code), and a write-up detailing your mini-project. You will be graded on these categories.

If you're OK with Python or R, please place all three parts into this notebook/.Rmd file that we have provided where indicated. We have written template sections for you to follow for simplicity/completeness. When you're done, save as a `.pdf` (please knit to `.pdf` if you are using `.Rmd`, or knit to `.html` and use a browser's "Print" function to convert to `.pdf`).

If you don't like Python OR R, we will allow you to use a different language, but please turn your assignment in with: 1) a folder with all your code, 2) a folder with all your figures, and 3) a `.tex`/`.doc`/`.pdf` file with a write-up.

## PROJECT TITLE HERE

**1. Describe what you are doing and why it matters to patients using at least one citation.**

In US, breast cancer is the second leading cause of cancer for women and it has been found that screening mammography could be incredibly helpful in terms of reducing mortality. However, this technique has been associated with a high risk of false positives as well as false negatives. Deep Learning tools have the potential to assist physicians and help patients in terms of diagnosing the patients, detecting tumor regions, as well as saving physicians' time. 


We plan to use the mammograms images along with its masks to predict tumor labels. We plan to start the project with a classic convolutional neural network to take the two images as inputs and return the diagnosis label. We will then calcualte its misclassification errors as well as AOC and ROC curves. If the model's accuracy does not match our expectations, we plan to explore other methods such as utilizing a pre-trained model. In the end, we will evaluate each model that we use and summarize the pros and cons of different training strategies. 


citation: https://www.nature.com/articles/s41598-019-48995-4

**2. Describe the relevant statistics of the data. How were the images taken? How were they labeled? What is the class balance and majority classifier accuracy? How will you divide the data into testing, training and validation sets?**

Mammogram images are taken using X-ray and they are typically taken from 2 angles. In our case, the data are taken in both the craniocaudal and mediolateral oblique views. The two main types of breast changes found with a mammogram are calcifications and masses. The label that we used is the pathology label from the mass_case_diagnosis from the mass_case_description_train_set dataset file. The three different classes are MALIGNANT, BENIGN, and BENIGN_WITHOUT_MASK. 

The class MALIGNANT appeared 305 times, the class BENIGN appeared 230 times, and the class BENIGN_WITHOUT_CALLBACK appeared 40 times. The majority classifier acccuracy is 0.53. 

We utilized the train_test_split function from the sklearn library to divide up the dataset into testing, training and validation sets. 

citation: https://www.cancer.org/cancer/breast-cancer/screening-tests-and-early-detection/mammograms/mammogram-basics.html

**3. Describe your data pipeline (how is the data scrubbed, normalized, stored, and fed to the model for training?).**

To obtain the input data, we traversed through all the scans under the directory "data", stored the corresponding patient_id, side and view for each image file taken. We stored the input X as a three dimensional pixel array. Then we used the patient_id, side and view features to find the corresponding patient in the mass_Case_description_train_set dataset, and extracted their pathology result. We were able to find 575 patients that exists both in the data file and the mass_case_description dataset this way. 

We used the centering method to rescale the input X. We subtract the mean pixel and then divide by the standard deviation. we hope that the centering method could help us eliminate vanishing and exploding gradients, and increase convergence speed and accuracy. 

We used the LabelBinarizer function from sklearn library to rescale the output y. We used the one hot encoding method to convert the three output classes of y into indivator vectors. 

**4. Explain how the model you chose works alongside the code for it. Add at least one technical citation to give credit where credit is due.**

In [49]:
import os
import numpy as np
import pydicom as dicom
import matplotlib.pyplot as plt
import os
import skimage
import sys
import h5py
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Conv3D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, UpSampling2D, concatenate
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras import Sequential
import cv2
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import datasets, layers, models

In [50]:
# preprocess data
file_root = "data"
case = pd.read_csv("mass_case_description_train_set.csv")
def get_data(path):
    X = []
    y = []
    path_contents = os.walk(path)
    for root, directories, files in path_contents: 
        for file in files:
            if (file.split(".")[1] == "h5"):
                path = root + '/' + file
                h5 = h5py.File(path, 'r')
                

                patient_id = "P_" + root.split("/")[1]
                side = file.split(".")[0].split("_")[0]
                view = file.split(".")[0].split("_")[1]
                pathology = case[(case.patient_id == patient_id) & (case.side == side) & (case.view == view) ]["pathology"]
                if (len(pathology.tolist())>0):
                    X.append(h5['data'])
                    pathology = pathology.tolist()[0]
                    y.append(pathology)
    X = np.array(X)
    y = np.array(y)
    return X, y

def normalize_data(x, y):
    x -= x.mean(axis=(1, 2), keepdims=True)
    x /= x.std(axis=(1, 2), keepdims=True)
    encoder = LabelBinarizer()
    transformed_label = encoder.fit(np.unique(y).tolist())
    y = transformed_label.transform(y.tolist())
    return x, y

We incorporated dropout layers into our model. Dropout layers have a regularizing effect and increase generalizability to new data. Dropout layers will not include a certain amount of nodes at each training iteration, which ensures new associations can be found.

Each convolutional layer in our model encodes a more abstract representation of the input images. The abstract features derived from the convolutional layers are analyzed by the Dense layers which determine which features are important for classifying tumor

We got the model from this website (https://www.analyticsvidhya.com/blog/2019/04/build-first-multi-label-image-classification-model-python/)

In [51]:
def model_1():
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(256,256,2)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='sigmoid'))
    print(model.summary())
    return model


In [ ]:
X, y = get_data(file_root)
X, y = normalize_data(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

model = model_1()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=64)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 252, 252, 16)      816       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 126, 126, 16)      0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 126, 126, 16)      0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 122, 122, 32)      12832     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 61, 61, 32)        0         
_________________________________________________________________
dropout_49 (Dropout)         (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 57, 57, 64)       

In [40]:
model.predict(X_test)

array([[9.42416191e-01, 1.08337402e-02, 6.92186952e-02],
       [1.53840959e-01, 1.33333206e-02, 8.33658099e-01],
       [3.91617239e-01, 2.19608545e-02, 6.04117930e-01],
       [5.07200122e-01, 3.20895910e-01, 4.19282258e-01],
       [4.25052345e-01, 3.51122022e-02, 5.33119678e-01],
       [4.85127002e-01, 2.11145580e-01, 3.80196154e-01],
       [3.07861626e-01, 5.66468537e-02, 6.86826825e-01],
       [5.11783540e-01, 9.92894173e-04, 4.31017578e-01],
       [2.98788607e-01, 4.78214025e-02, 7.16281056e-01],
       [3.28314126e-01, 3.34119201e-02, 5.87500453e-01],
       [2.81976819e-01, 2.23943293e-02, 6.22652888e-01],
       [5.21813929e-01, 1.62944496e-02, 5.01733303e-01],
       [4.39897269e-01, 4.44793403e-02, 5.71913779e-01],
       [8.83205712e-01, 1.92320347e-02, 1.06046677e-01],
       [7.12188244e-01, 2.59187818e-01, 1.54484361e-01],
       [4.29615945e-01, 1.27773792e-01, 5.49730539e-01],
       [4.14453655e-01, 1.84551418e-01, 5.67775905e-01],
       [3.77051771e-01, 3.97221

In [44]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

lr_probs = model.predict_proba(X_test)
#ns_probs = []
lr_probs = lr_probs[:, 1]
#ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)

from sklearn.metrics import confusion_matrix

/home/users/hzhu04/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


ValueError: Expected 2D array, got 1D array instead:
array=[1.08337402e-02 1.33333206e-02 2.19608545e-02 3.20895910e-01
 3.51122022e-02 2.11145580e-01 5.66468537e-02 9.92894173e-04
 4.78214025e-02 3.34119201e-02 2.23943293e-02 1.62944496e-02
 4.44793403e-02 1.92320347e-02 2.59187818e-01 1.27773792e-01
 1.84551418e-01 3.97221744e-01 6.77354038e-02 4.85221744e-02
 1.90427303e-02 8.84267688e-03 1.24444962e-02 2.26965845e-02
 2.01837659e-01 5.91087341e-02 1.10010803e-02 9.56961513e-03
 5.13283014e-02 6.95422292e-03 2.13678181e-02 7.71701336e-03
 4.77204621e-02 1.79794431e-03 2.11091042e-02 2.62201726e-02
 1.16011053e-01 1.11631155e-02 3.53227258e-01 4.48843837e-03
 3.93179059e-02 8.26039910e-02 4.00781631e-04 2.67290950e-01
 1.27239972e-01 3.45009565e-02 2.90876597e-01 8.10965896e-03
 4.39710349e-01 3.43884826e-02 1.49853230e-02 5.03167510e-03
 8.34847093e-02 1.70892477e-03 3.17543745e-04 9.72315669e-03
 8.92082751e-02 7.59509206e-03].
Reshape your data either using array.reshape(-1, 1) if

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
class_namese= ["BENIGN", "MALIGNANT", "BENIGN_WITHOUT"]
for title, normalize in titles_options:
    disp = plot_confusion_matrix( model, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

In [ ]:
# YOUR CODE FOR QUESTION 4 HERE. USE ADDITIONAL CODE/MARKDOWN CELLS IF NEEDED

YOUR WRITTEN ANSWER TO QUESTION 4 HERE. USE ADDITIONAL CODE/MARKDOWN CELLS IF NEEDED

**5. There are many ways to do training. Take us through how you do it (e.g. "We used early stopping and stopped when validation loss increased twice in a row.").**

In order to train our model, we use 50 training epochs. Moreover, we split the data into a train set (66%) and a test set (33%) to ensure our evaluation is not susceptible to overfitting and is representative of new input. This training and evaluation procedure allowed our model to have success in learnign how to predict tumors and also a fair evaluation

**6. Make a figure displaying your results.**

In [ ]:
# YOUR CODE FOR QUESTION 6 HERE

YOUR WRITTEN ANSWER TO QUESTION 6 HERE

**7. Discuss pros and cons of your method and what you might have done differently now that you've tried or would try if you had more time.**

The main pro of our methodology is that we are using a deep learning network that is quite powerful and should be able to extract all the features pertinent to tumor identification. Moreover, we are using classically succesful pre-processing techniques like normalization, and regularization techniques like dropout layers. This general procedure has been effective in other similar tasks and so we think this is a reasonable strategy. This is shown by our relatively succesful accuracy of X %.

Cons: Our dataset is too small for this neural network. 

There are several areas that we can improve on. First, our model is not comprehensively tuned, and it would be beneficial to test more hyper-parameters such as batch sizes, epochs, optimizer, number of layers, etc.... Our model also has a rather modest data set with 175 patients. In order to achieve state-of-the-art results, a dataset that is far larger would be required. While normalizing is a good first step for pre-processing, we could also apply other techniques such as gaussian smoothing, data augmentation, and housfield unit conversion. Using pre-trained networks is also a strategy that has shown classical success that we did not try. In general, it seems as though our strategy is good, but would benefit from more trial-and-error with equally good ideas

**You will not be graded on the performance of your model. You'll only be graded on the scientific soundness of your claims, methodology, evaluation (i.e. fair but insightful statistics), and discussion of the strengths and shortcomings of what you tried. Feel free to reuse some of the code you are/will be using for your projects. The write-up doesn't need to be long (~1 page will suffice), but please cite at least one clinical paper and one technical paper (1 each in questions 1 and 4 at least, and more if needed).**